In [ ]:
import pandas as pd
import ollama
from datetime import timedelta

In [ ]:
def preprocess(df):
    df = df[df['created_at'] <= '2024-04-24 17:37']
    # 1 bis 6
    # keep only rows where 'user_name' includes 'ths3'
    #df = df[df['user_name'].str.contains('ths3', case=False)]
    df = df[df['user_name'].str.contains(r'ths3p[1-6]', case=False)]
    # if two rows have same session id and same answers to all questions, keep only one row
    df.drop_duplicates(subset=['session_id', 'yrActivities', 'onYrMind'], keep='first', inplace=True)

    # drop rows where the session id is longer than 5 digits
    df = df[df['session_id'].astype(str).map(len) <= 5]
    
    # if two rows have the same session id but different answers to the questions, add one day to the date in "created_at" column
    # Convert created_at to datetime
    df['created_at'] = pd.to_datetime(df['created_at'])
    
    # Identify duplicates by session_id
    duplicates = df[df.duplicated(subset=['session_id'], keep=False)]
    
    # Iterate over each group of duplicates
    for session_id, group in duplicates.groupby('session_id'):
        if len(group) > 1:
            # Sort by created_at to add days incrementally
            group = group.sort_values(by='created_at')
            for i in range(1, len(group)):
                df.loc[group.index[i], 'created_at'] += timedelta(days=i)
                
    df = df.dropna(subset=['yrDay'])
    
    return df

# Prepare dfs
df = pd.read_csv('data/pepper_dump_19_06_2024.csv')

df_eval_c = pd.read_csv('data/peppers_manually_labeled_response_redacted.csv')
df_eval_c['created_at'] = pd.to_datetime(df_eval_c['created_at'], errors='coerce')
df_eval_c['created_at'] = df_eval_c['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')

# merge all cols to df_eval_c on FsDaily_id
additional_columns = [col for col in df.columns if col not in df_eval_c.columns]
df_eval_c = df_eval_c.merge(df[['FsDaily_id'] + additional_columns], on='FsDaily_id', how='inner')

# Apply preprocessing
df = preprocess(df)
df_eval = preprocess(df_eval_c)
df

In [ ]:
# Mapping für die Sentiment-Werte
sentiment_map = {1: 'positiv', 0: 'neutral', -1: 'negativ'}

def map_sentiment(value):
    return sentiment_map[value]

# Erstelle manual_labels DataFrame mit manuell gelabelten Sentiments
manual_labels = pd.DataFrame({
    'FsDaily_id': df_eval['FsDaily_id'],
    'manual_label_yrDay': df_eval['manual_label_yrDay'].apply(map_sentiment),
    'manual_label_yrActivities': df_eval['manual_label_yrActivities'].apply(map_sentiment),
    'manual_label_madeYouHappy': df_eval['manual_label_madeYouHappy'].apply(map_sentiment),
    'manual_label_onYrMind': df_eval['manual_label_onYrMind'].apply(map_sentiment),
})

manual_labels

In [ ]:
def extract_label(response):
    candidate_labels = ["positiv", "neutral", "negativ"]

    for label in candidate_labels:
        if label in response:
            return label
    return None

def analyze_sentiment(df, column):
    if column == "yrDay":
        prompt_template = ("""
                Bitte analysiere den folgenden Text nach Sentiment und ordne ihn einer der vorgegebenen Kategorien zu: 'positiv', 'neutral', 'negativ'.
                
                Den Nutzern wurde dieser Text als Frage gestellt: Wie war dein tag? Darauf antworten die Nutzer mit den Dingen, wie ihr Tag war.
                
                Bitte analysiere die Antwort und weise die passendste Kategorie zu. Hier sind die Regeln für die Zuordnung:
                
                - 'positiv': Wenn der Text eine Freude, Zufriedenheit oder ein glückliches Ereignis ausdrückt.
                - 'neutral': Wenn der Text eine neutrale Aussage oder Information enthält, ohne starke emotionale Ausdrucksweise.
                - 'negativ': Wenn der Text Unzufriedenheit, Frustration oder ein unglückliches Ereignis ausdrückt.
                - Falls der Text keine Informationen enthält, die eine genaue Zuordnung zulassen, antworte mit 'None'.
                
                Die Länge des Texts gibt keinen Aufschluss über das Sentiment.
                
                Schau dir die Frage und die Antwort genau an und weiße eine der Kategorien zu. Beachte dabei, dass die Frage ist WIE WAR DEIN TAG?
                
                Frage: Wie war dein Tag?
                Antwort: {}
                
                Gib NUR das passende Label zurück, ohne zusätzliche Formatierungen oder Erklärungen.


            """)
        system_message = {
        'role': 'system',
        'content': ("Du bist ein Bewerter, der Texte in Kategorien einteilt. Deine Aufgabe ist es, "
                    "jeden gegebenen Text genau zu analysieren und das passendste Label auszuwählen. "
                    "Es geht darum, das Sentiment zu bestimmen. Beachte, dass die Frage darauf abzielt, "
                    ""),
                        }
    elif column == "onYrMind":
        prompt_template = ("""
                Bitte analysiere den folgenden Text nach Sentiment und ordne ihn einer der vorgegebenen Kategorien zu: 'positiv', 'neutral', 'negativ'.
                
                Den Nutzern wurde dieser Text als Frage gestellt: Was hat dich beschäftigt? Darauf antworten die Nutzer mit den Dingen, die sie beschäftigt haben.
                
                Bitte analysiere die Antwort und weise die passendste Kategorie zu. Hier sind die Regeln für die Zuordnung:
                
                - 'positiv': Wenn der Text eine Freude, Zufriedenheit oder ein glückliches Ereignis ausdrückt.
                - 'neutral': Wenn der Text eine neutrale Aussage oder Information enthält, ohne starke emotionale Ausdrucksweise.
                - 'negativ': Wenn der Text Unzufriedenheit, Frustration oder ein unglückliches Ereignis ausdrückt.
                - Falls der Text keine Informationen enthält, die eine genaue Zuordnung zulassen, antworte mit 'None'.
                
                Die Länge des Texts gibt keinen Aufschluss über das Sentiment.
                
                Schau dir die Frage und die Antwort genau an und weiße eine der Kategorien zu. Beachte dabei, dass die Frage ist Was hat dich beschäftigt?
                
                Frage: Was hat dich beschäftigt?
                Antwort: {}
                
                Gib NUR das passende Label zurück, ohne zusätzliche Formatierungen oder Erklärungen.


            """)
        system_message = {
        'role': 'system',
        'content': ("Du bist ein Bewerter, der Texte in Kategorien einteilt. Deine Aufgabe ist es, "
                    "jeden gegebenen Text genau zu analysieren und das passendste Label auszuwählen. "
                    "Es geht darum, das Sentiment zu bestimmen. Beachte, dass die Frage darauf abzielt, "
                    ),
                        }
    
    elif column == "madeYouHappy":
        prompt_template = ("""
                Bitte analysiere den folgenden Text nach Sentiment und ordne ihn einer der vorgegebenen Kategorien zu: 'positiv', 'neutral', 'negativ'.
                
                Den Nutzern wurde dieser Text als Frage gestellt: Was hat dich glücklich gemacht? Darauf antworten die Nutzer mit den Dingen, die sie glücklich gemacht haben.
                
                Bitte analysiere die Antwort und weise die passendste Kategorie zu. Hier sind die Regeln für die Zuordnung:
                
                - 'positiv': Wenn der Text eine Freude, Zufriedenheit oder ein glückliches Ereignis ausdrückt.
                - 'neutral': Wenn der Text eine neutrale Aussage oder Information enthält, ohne starke emotionale Ausdrucksweise.
                - 'negativ': Wenn der Text Unzufriedenheit, Frustration oder ein unglückliches Ereignis ausdrückt.
                - Falls der Text keine Informationen enthält, die eine genaue Zuordnung zulassen, antworte mit 'None'.
                
                Die Länge des Texts gibt keinen Aufschluss über das Sentiment.
                
                Schau dir die Frage und die Antwort genau an und weiße eine der Kategorien zu. Beachte dabei, dass die Frage ist WAS HAT DICH GLÜCKLICH GEMACHT!
                Frage: Was hat dich glücklich gemacht?
                Antwort: {}
                
                Gib NUR das passende Label zurück, ohne zusätzliche Formatierungen oder Erklärungen.


            """)
        system_message = {
        'role': 'system',
        'content': ("Du bist ein Bewerter, der Texte in Kategorien einteilt. Deine Aufgabe ist es, "
                    "jeden gegebenen Text genau zu analysieren und das passendste Label auszuwählen. "
                    "Es geht darum, das Sentiment zu bestimmen. Beachte, dass die Frage darauf abzielt, "
                    "Dinge zu erfahren, die die Nutzer glücklich gemacht haben."),
                        }
    
    
    elif column == "yrActivities":
        prompt_template = ("""
                Bitte analysiere den folgenden Text nach Sentiment und ordne ihn einer der vorgegebenen Kategorien zu: 'positiv', 'neutral', 'negativ'.
                
                Den Nutzern wurde dieser Text als Frage gestellt: Was hast du unternommen? Darauf antworten die Nutzer mit den Dingen, die sie getan haben.
                
                Bitte analysiere die Antwort und weise die passendste Kategorie zu. Hier sind die Regeln für die Zuordnung:
                
                1. **Positiv:** Die Antwort drückt explizit Freude, Zufriedenheit oder positive Emotionen aus. Beispiele:
                   - "Ich hatte einen großartigen Tag."
                   - "Ich habe eine schöne Zeit mit Freunden verbracht."
                   - "Ich habe das Golfspielen sehr genossen."
                
                2. **Neutral:** Die Antwort beschreibt lediglich eine Tätigkeit oder Handlung, ohne emotionale Wertung. Beispiele:
                   - "Ich habe geschlafen."
                   - "Ich war einkaufen."
                   - "Ich habe gearbeitet."
                   - "Ich habe Golf gespielt."
                
                
                3. **Negativ:** Die Antwort drückt EXPLIZIT Unzufriedenheit, Frustration oder NEGATIVE Emotionen aus. Beispiele:
                   - "Ich hatte einen schrecklichen Tag."
                   - "Ich war sehr enttäuscht."
                   - "Ich habe mich gelangweilt."
                   - "Das Golfspielen war furchtbar."
                   - "Ich fühle mich schlecht, weil ich nichts gemacht habe."
                
                4. Falls der Text keine Informationen enthält, die eine genaue Zuordnung zulassen, antworte mit 'None'.
                
                Die Länge des Texts gibt keinen Aufschluss über das Sentiment.
                
                Gib NUR das passende Label zurück, ohne zusätzliche Formatierungen oder Erklärungen.
                
                Text: {}
            """)
    
        system_message = {
                'role': 'system',
                'content': ("Du bist ein Bewerter, der Texte in Kategorien einteilt. Deine Aufgabe ist es, "
                            "jeden gegebenen Text genau zu analysieren und das passendste Label auszuwählen. "
                            ),
                         }
    
    
    classifications = []

    for sentence in df[column].fillna(''):
        current_prompt = prompt_template.format(sentence)

        response = ollama.chat(model='llama3', messages=[
                system_message,
                {
                    'role': 'user',
                    'content': current_prompt,
                },
            ],
            options = {
          #'temperature': 1.5, # very creative
          'temperature': 0
        }) 

        label = extract_label(response['message']['content'])
        classifications.append(label)

    df[column + '_classification'] = classifications

    return df

In [ ]:
res_yrDay = analyze_sentiment(df, column='yrDay')
res_selected_res_yrDay = res_yrDay[['yrDay', 'yrDay_classification']]
res_selected_res_yrDay

In [ ]:
df_merged = pd.merge(res_yrDay[['FsDaily_id', 'yrDay_classification']], 
                     manual_labels[['FsDaily_id', 'manual_label_yrDay']], 
                     on='FsDaily_id', how='inner')

def calculate_accuracy(predictions, labels):
    return (predictions == labels).mean()

acc_yrDay = calculate_accuracy(df_merged['yrDay_classification'], df_merged['manual_label_yrDay'])

print(f"Genauigkeit yrDay: {acc_yrDay:.2f}")

In [ ]:
res_onYrMind = analyze_sentiment(df, column='onYrMind')
res_selected_res_onYrMind = res_onYrMind[['onYrMind', 'onYrMind_classification']]
res_selected_res_onYrMind

In [ ]:
df_merged = pd.merge(res_yrDay[['FsDaily_id', 'onYrMind_classification']], 
                     manual_labels[['FsDaily_id', 'manual_label_onYrMind']], 
                     on='FsDaily_id', how='inner')

def calculate_accuracy(predictions, labels):
    return (predictions == labels).mean()

acc_onYrMind = calculate_accuracy(df_merged['onYrMind_classification'], df_merged['manual_label_onYrMind'])

print(f"Acc onYrMind: {acc_onYrMind:.2f}")

In [ ]:
negative_rows = res_selected_res_onYrMind[res_selected_res_onYrMind['onYrMind_classification'] == 'negativ']
negative_rows

In [ ]:
res_madeYouHappy = analyze_sentiment(df, column='madeYouHappy')
res_selected_res_madeYouHappy = res_madeYouHappy[['madeYouHappy', 'madeYouHappy_classification']]
res_selected_res_madeYouHappy

In [ ]:
df_merged = pd.merge(res_madeYouHappy[['FsDaily_id', 'madeYouHappy_classification']], 
                     manual_labels[['FsDaily_id', 'manual_label_madeYouHappy']], 
                     on='FsDaily_id', how='inner')

def calculate_accuracy(predictions, labels):
    return (predictions == labels).mean()

acc_yrDay = calculate_accuracy(df_merged['madeYouHappy_classification'], df_merged['manual_label_madeYouHappy'])

print(f"Acc madeYouHappy: {acc_yrDay:.2f}")

In [ ]:
negative_rows = res_selected_res_madeYouHappy[res_selected_res_madeYouHappy['madeYouHappy_classification'] == 'negativ']
negative_rows

In [ ]:
# res_yr_activities
res_yr_activities = analyze_sentiment(df, column='yrActivities')
res_selected_res_yr_activities = res_yr_activities[['yrActivities', 'yrActivities_classification']]
res_selected_res_yr_activities

In [ ]:
df_merged = pd.merge(res_yr_activities[['FsDaily_id', 'yrActivities_classification']], 
                     manual_labels[['FsDaily_id', 'manual_label_yrActivities']], 
                     on='FsDaily_id', how='inner')

def calculate_accuracy(predictions, labels):
    return (predictions == labels).mean()

acc_yrActivities = calculate_accuracy(df_merged['yrActivities_classification'], df_merged['manual_label_yrActivities'])

print(f"Acc yrActivities: {acc_yrActivities:.2f}")